In [66]:
import numpy as np
import pandas as pd
from collections import Counter

def read(train_data,test_data):
    train_data = pd.read_csv(train_data)
    test_data = pd.read_csv(test_data)
    return train_data,test_data

In [67]:
def data_process(train_data,test_data):
    train_data.replace({'low':0,'med':1,'high':2,'vhigh':3,'5more':5,'small':0,'big':2,'more':5,'2':2,'3':3,'4':4, 'unacc':'1', 'acc':'0', 'good':'-1'},inplace=True)
    test_data.replace({'low':0,'med':1,'high':2,'vhigh':3,'5more':5,'small':0,'big':2,'more':5,'2':2,'3':3,'4':4, 'unacc':'1', 'acc':'0', 'good':'-1'},inplace=True)
    return train_data,test_data

In [68]:
def split(train_data,test_data):
    train_set = train_data.sample(frac=0.8)
    validation_set = train_data[~train_data.index.isin(train_set.index)]
    #f is feature, l is label
    f1 = train_set.iloc[:, [0,1,2,3,4,5]].values
    l1 = train_set.iloc[:, 6].values
    f2 = validation_set.iloc[:, [0,1,2,3,4,5]].values
    l2 = validation_set.iloc[:, 6].values
    f3 = test_data.iloc[:, [0,1,2,3,4,5]].values
    return f1,l1,f2,l2,f3

In [69]:
train_data, test_data=read('Dataset-20221118/training.csv','Dataset-20221118/test.csv')
train_data, test_data=data_process(train_data,test_data)
f1,l1,f2,l2,f3=split(train_data,test_data)

In [78]:
import pandas as pd
import numpy as np
#数据集文件路径
 
#获取训练集（原始训练集百分之八十）、验证集（原始训练集百分之二十）、测试集
# def getData(filepath):
#     df_train = pd.read_excel(filepath, sheet_name='training')
#     df_test = pd.read_excel(filepath, sheet_name='test')
#     length = len(df_train.values)
#     x_train = df_train.values[:int(0.8 * length), :-1]
#     y_train = df_train.values[:int(0.8 * length), -1]
#     x_val = df_train.values[int(0.8 * length):, :-1]
#     y_val = df_train.values[int(0.8 * length):, -1]
#     x_test = df_test.values[:, :-1]
#     return x_train, y_train, x_val, y_val, x_test
def split(train_data,test_data):
    train_set = train_data.sample(frac=0.8)
    validation_set = train_data[~train_data.index.isin(train_set.index)]
    #x is feature, y is label
    x_train = train_set.iloc[:, [0,1,2,3,4,5]].values
    y_train = train_set.iloc[:, 6].values
    x_val = validation_set.iloc[:, [0,1,2,3,4,5]].values
    y_val = validation_set.iloc[:, 6].values
    x_test = test_data.iloc[:, [0,1,2,3,4,5]].values
    return x_train, y_train, x_val, y_val, x_test
 
 
def main():
    #学习率
    #lr = 0.000001
    lr = 0.000001
    # 类别一维转三维
    classMap = {'-1': [1, 0, 0],
                '0': [0, 1, 0],
                '1': [0, 0, 1]}
    #类别映射
    class_map = [-1, 0, 1]
    x_train, y_train, x_val, y_val, x_test = split(train_data,test_data)
    #随机初始化W、b
    W = np.random.randn(3, 6)
    b = np.random.randn(3)
    #训练6000次
    for i in range(6000):
        loss = 0
        #初始化偏导
        alpha1 = [0] * 6
        alpha2 = [0] * 6
        alpha3 = [0] * 6
        beta1 = 0
        beta2 = 0
        beta3 = 0
        for xi, yi in zip(x_train, y_train):
            ai = np.sum(np.multiply([xi] * 3, W), axis=1) + b
            y_predicti = np.exp(ai) / sum(np.exp(ai))
            y_i = classMap[str(yi)]
            lossi = -sum(np.multiply(y_i, np.log(y_predicti)))
            loss += lossi
            # 每个训练数据偏导累加
            alpha1 += np.multiply(sum(np.multiply([0, 1, 1], y_i)), xi)
            alpha2 += np.multiply(sum(np.multiply([1, 0, 1], y_i)), xi)
            alpha3 += np.multiply(sum(np.multiply([1, 1, 0], y_i)), xi)
            beta1 += sum(np.multiply([0, 1, 1], y_i))
            beta2 += sum(np.multiply([1, 0, 1], y_i))
            beta3 += sum(np.multiply([1, 1, 0], y_i))
        #W、b更新值
        W[0] -= alpha1 * lr
        W[1] -= alpha2 * lr
        W[2] -= alpha3 * lr
        b[0] -= beta1 * lr
        b[1] -= beta2 * lr
        b[2] -= beta3 * lr
        loss = loss/len(x_train)
    recall = 0
    # #验证
    # for xi, yi in zip(x_val, y_val):
    #     ai = np.sum(np.multiply([xi] * 3, W), axis=1) + b
    #     y_predicti = np.exp(ai) / sum(np.exp(ai))
    #     y_predicti = [class_map[idx] for idx, i in enumerate(y_predicti) if i == max(y_predicti)][0]
    #     recall += 1 if int(y_predicti) == yi else 0
    # print('验证集总条数：', len(x_val), '预测正确数：', recall)
    # fp = open('test.csv', 'w')
    for xi, yi in zip(x_val, y_val):
        ai = np.sum(np.multiply([xi] * 3, W), axis=1) + b
        y_predicti = np.exp(ai) / sum(np.exp(ai))
        y_predicti = [class_map[idx] for idx, i in enumerate(y_predicti) if i == max(y_predicti)][0]
        recall += 1 if int(y_predicti) == int(yi) else 0
    print('验证集总条数：', len(x_val), '预测正确数：', recall)
    #测试
    # for xi in x_test:
    #     ai = np.sum(np.multiply([xi] * 3, W), axis=1) + b
    #     y_predicti = np.exp(ai) / sum(np.exp(ai))
    #     y_predicti = [class_map[idx] for idx, i in enumerate(y_predicti) if i == max(y_predicti)][0]
    #     fp.write(str(y_predicti)+'\n')
    # fp.close()

if __name__ == '__main__':
    print('方法三：感知机')
    main()

方法三：感知机
验证集总条数： 266 预测正确数： 200


In [46]:
df_train, df_test=read('Dataset-20221118/training.csv','Dataset-20221118/test.csv')
df_train, df_test=data_process(df_train, df_test)
length = len(df_train.values)
x_train = df_train.values[:int(0.8 * length), :-1]
y_train = df_train.values[:int(0.8 * length), -1]
x_val = df_train.values[int(0.8 * length):, :-1]
y_val = df_train.values[int(0.8 * length):, -1]
x_test = df_test.values[:, :-1]
print(y_train)

['1' '1' '1' ... '1' '0' '1']


In [74]:
lr = 0.000001
# 类别一维转三维
classMap = {'-1': [1, 0, 0],
            '0': [0, 1, 0],
            '1': [0, 0, 1]}
#类别映射
class_map = [-1, 0, 1]
x_train, y_train, x_val, y_val, x_test = split(train_data,test_data)
#随机初始化W、b
W = np.random.randn(3, 6)
b = np.random.randn(3)
#训练6000次
#for i in range(6000):
loss = 0
#初始化偏导
alpha1 = [0] * 6
alpha2 = [0] * 6
alpha3 = [0] * 6
beta1 = 0
beta2 = 0
beta3 = 0
for xi, yi in zip(x_train, y_train):
    # print(xi,yi)
    # print('\n')
    # print(x_train)
    # print('\n')
    # print(y_train)
    # print('\n')
    ai = np.sum(np.multiply([xi] * 3, W), axis=1) + b
    y_predicti = np.exp(ai) / sum(np.exp(ai))
    y_i = classMap[str(yi)]
    lossi = -sum(np.multiply(y_i, np.log(y_predicti)))
    loss += lossi
    # 每个训练数据偏导累加
    alpha1 += np.multiply(sum(np.multiply([0, 1, 1], y_i)), xi)
    alpha2 += np.multiply(sum(np.multiply([1, 0, 1], y_i)), xi)
    alpha3 += np.multiply(sum(np.multiply([1, 1, 0], y_i)), xi)
    beta1 += sum(np.multiply([0, 1, 1], y_i))
    beta2 += sum(np.multiply([1, 0, 1], y_i))
    beta3 += sum(np.multiply([1, 1, 0], y_i))